In [31]:
import pandas as pd
import sqlite3
import re
from contextlib import closing
from datetime import datetime

In [36]:
db = "test.db"

query_sessions = '''
SELECT name as sessions,name as events FROM
(
SELECT * FROM
	(SELECT name FROM sqlite_master WHERE name LIKE 'session_%'),
	(SELECT name FROM sqlite_master WHERE name LIKE 'events_%')
)
'''

def get_uid(name:str)->str:
    return re.findall(r"\d+", name)

def get_date(name:str)->datetime:
    return datetime.strptime(
        re.findall(r"\d+", name)[0],
        r'%Y%m%d%H%M%S'
    ).strftime(r"%c")
    
with closing(sqlite3.connect(db)) as conn:
    sessions = pd.read_sql(query_sessions, conn)

    print(get_date(sessions['sessions'][0]))

Thu Jul 18 20:59:00 2024
